In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import train,prepare,evaluate,tune
from models.Interactors import FIM_Interactor, KNRM_Interactor
from models.Encoders.FIM import FIM_Encoder
from models.NRMS import NRMS_MultiView
from models.SFI import SFI_gating_MultiView,SFI_ensemble
from configs.ManualConfig import hparams

In [2]:
hparams['mode'] = 'dev'
hparams['onehot'] = True
hparams['k'] = 30
hparams['ensemble'] = 'softmax'
# hparams['device'] = 'cuda:0'

In [3]:
# hparams['validate'] = True
vocab, loaders = prepare(hparams, pin_memory=False)

[2021-04-18 16:19:28,910] INFO (root) Hyper Parameters are
{'scale': 'demo', 'mode': 'dev', 'batch_size': 10, 'title_size': 20, 'abs_size': 40, 'his_size': 50, 'vert_num': 18, 'subvert_num': 293, 'npratio': 4, 'dropout_p': 0.2, 'query_dim': 200, 'embedding_dim': 300, 'filter_num': 150, 'value_dim': 16, 'head_num': 16, 'epochs': 8, 'metrics': 'auc,mean_mrr,ndcg@5,ndcg@10', 'device': 'cpu', 'attrs': ['title'], 'k': 30, 'select': None, 'save_step': [0], 'news_id': False, 'validate': False, 'interval': 10, 'spadam': True, 'name': 'sfi-ensemble', 'onehot': True, 'ensemble': 'softmax'}
[2021-04-18 16:19:28,912] INFO (root) preparing dataset...
[2021-04-18 16:19:31,868] INFO (torchtext.vocab) Loading vectors from .vector_cache/glove.840B.300d.txt.pt


In [4]:
record = next(iter(loaders[0]))

In [5]:
encoder = FIM_Encoder(hparams, vocab)
interactor = FIM_Interactor()

nrms = NRMS_MultiView(hparams, vocab).to(hparams['device'])
sfi = SFI_gating_MultiView(hparams, encoder, interactor).to(hparams['device'])

sfi_ensemble = SFI_ensemble(hparams, [sfi, nrms])

[2021-04-18 16:19:37,486] INFO (models.SFI) automatically adjust cdd_size of each model to 1


In [6]:
sfi(record)

tensor([[0.4762],
        [0.5852],
        [0.6025],
        [0.2844],
        [0.5233],
        [0.5901],
        [0.5223],
        [0.6681],
        [0.6383],
        [0.4943]], grad_fn=<SigmoidBackward>)

In [7]:
evaluate(sfi_ensemble, hparams, loaders[0])

[2021-04-18 16:19:37,883] INFO (root) this model is not inherited from nn.Module
[2021-04-18 16:19:37,885] INFO (root) evaluating...
  2%|▏         | 33/1812 [00:16<15:08,  1.96it/s]